# Machine Learning: Text Classification Assignment

In [7]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [10]:
!python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [8]:
import spacy
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split as tts
from nltk.corpus.reader.plaintext import CategorizedPlaintextCorpusReader

In [9]:
import nltk

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Use the CategorizedPlaintextCorpusReader to import the AP_News corpus.

In [20]:
DOC_PATTERN = r".*\.txt"
CAT_PATTERN = r"([\w_\s]+)/.*"
path = "./AP_News/"

corpus = CategorizedPlaintextCorpusReader(path, DOC_PATTERN,
                                          cat_pattern=CAT_PATTERN)

### Create two separate lists - one containing the text from each document and another containing the category of each article in the corpus.

In [21]:
docs = [corpus.raw(id_) for id_ in corpus.fileids()]
categories = [corpus.categories(id_)[0] for id_ in corpus.fileids()]

### Preprocess the corpus, ensuring to include the following steps.

- Word tokenize the documents.
- Lemmatize, stem, and lowercase all tokens.
- Remove punctuation and stop words.

In [22]:
def preprocess(docs):
    lemmatizer = WordNetLemmatizer()
    stemmer = SnowballStemmer("english")
    preprocessed = []

    for doc in docs:
        tokenized = word_tokenize(doc)
        cleaned = [stemmer.stem(lemmatizer.lemmatize(token.lower()))
        for token in tokenized
        if not token.lower() in stopwords.words("english")
        if token.isalpha()]

        untokenized = " ".join(cleaned)
        preprocessed.append(untokenized)

    return preprocessed

In [23]:
processed_docs = preprocess(docs)

### Split the data into training and testing sets with the size of the test set being 30% of the records.

In [30]:
from sklearn.model_selection import train_test_split as tts

X_train, X_test, y_train, y_test = tts(processed_docs, categories, test_size=0.2)

### Construct a pipeline that TF-IDF vectorizes the text and trains a Random Forest classification model.

In [31]:
from sklearn.linear_model import LogisticRegression

model = Pipeline(
    [
     ('vect', CountVectorizer()),
     ('tfidf', TfidfTransformer()),
     ('clf', LogisticRegression())
    ])

model.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
  

### Generate predictions on the test set and print a classification report to evaluate how well the model performed.

In [32]:
pred = model.predict(X_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

      health       0.88      0.70      0.78        10
    politics       0.56      0.90      0.69        10
      sports       1.00      0.69      0.82        13
        tech       0.64      0.64      0.64        11

    accuracy                           0.73        44
   macro avg       0.77      0.73      0.73        44
weighted avg       0.78      0.73      0.73        44



### Perform 10-fold cross validation and obtain the averge F1 score across all the folds.

In [33]:
scores = cross_val_score(model, processed_docs, categories, cv=10, scoring='f1_macro')
scores.mean()

0.8187741262005968

### Ingest, preprocess, and predict the topic of the article at the following URL.

In [34]:
url = 'https://www.nytimes.com/2019/11/25/business/uber-london.html'

In [37]:
import requests
from bs4 import BeautifulSoup

def get_url_text(url):
    response = requests.get(url)
    content = response.text

    TAGS = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'h7', 'p', 'li']
    soup = BeautifulSoup(content, "lxml")
    text_list = [tag.get_text() for tag in soup.find_all(TAGS)]
    text = " ".join(text_list)
    return text

text = get_url_text(url)
cleaned = preprocess([text])
model.predict(cleaned)[0]

'tech'